# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
pip install yfinance cvxpy

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 28.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 56.4 MB/s eta 0:00:00:00:01
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=122cfa0b379c825cebd1ed048c76eb1bcd5e887866232a72997f540aaaee57cb
  Stored in directory: /home/codespace/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.2-cp

In [6]:
# YOUR CHANGES HERE
import pandas as pd, yfinance as yf

tickers = ["AAPL","MSFT","AMZN","GOOGL","SPY","TSLA"]

# Daily adjusted close
px = yf.download(tickers, period="3y", interval="1d", auto_adjust=False)
px = px["Adj Close"] if isinstance(px.columns, pd.MultiIndex) else px
px = px.dropna(how="all")

# Resample to last trading day of each month
month_end = px.resample("M").last()

# Cut off at last Friday (replace date string with the Friday you mean, e.g., 2025-08-22)
cutoff = "2025-08-22"
month_end = month_end.loc[:cutoff]

[*********************100%***********************]  6 of 6 completed
/tmp/ipykernel_3000/1586181500.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_end = px.resample("M").last()


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [8]:
# YOUR CHANGES HERE

month_end.index.name = "date"
month_end = month_end.reindex(columns=tickers)  # enforce order
month_end.to_csv("historical_prices.tsv", sep="\t",
                 float_format="%.2f", date_format="%Y-%m-%d")

print("Saved historical_prices.tsv through", cutoff)
print(["date"] + month_end.columns.tolist())


Saved historical_prices.tsv through 2025-08-22
['date', 'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'SPY', 'TSLA']


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [10]:
# YOUR CHANGES HERE
import pandas as pd, numpy as np
df = pd.read_csv("historical_prices.tsv", sep="\t", parse_dates=["date"]).set_index("date")
ret = df.pct_change().dropna()
ret.tail()

mu  = ret.mean()                 # expected monthly return
sig = ret.std(ddof=1)            # monthly risk (std dev)
Sigma = ret.cov()                # covariance
Corr  = ret.corr()               # correlation
mu, sig, Corr.round(3)


(AAPL     0.010635
 MSFT     0.023264
 AMZN     0.021134
 GOOGL    0.019760
 SPY      0.015653
 TSLA     0.019515
 dtype: float64,
 AAPL     0.068217
 MSFT     0.065174
 AMZN     0.085074
 GOOGL    0.079328
 SPY      0.043355
 TSLA     0.183441
 dtype: float64,
         AAPL   MSFT   AMZN  GOOGL    SPY   TSLA
 AAPL   1.000  0.436  0.459  0.375  0.648  0.443
 MSFT   0.436  1.000  0.596  0.529  0.662  0.232
 AMZN   0.459  0.596  1.000  0.686  0.600  0.626
 GOOGL  0.375  0.529  0.686  1.000  0.481  0.384
 SPY    0.648  0.662  0.600  0.481  1.000  0.360
 TSLA   0.443  0.232  0.626  0.384  0.360  1.000)

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [11]:
# YOUR CHANGES HERE

# Save relative monthly returns to TSV
ret = ret.tail(23)   # keep 23 rows (24 prices → 23 returns)
ret.index.name = "date"
ret.to_csv("historical_returns.tsv", sep="\t",
           float_format="%.6f", date_format="%Y-%m-%d")

print("Saved historical_returns.tsv with shape:", ret.shape)
print(["date"] + ret.columns.tolist())

Saved historical_returns.tsv with shape: (23, 6)
['date', 'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'SPY', 'TSLA']


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [12]:
# YOUR CHANGES HERE

estimated_returns = ret.mean().reset_index()
estimated_returns.columns = ["asset", "estimated_return"]

Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [13]:
# YOUR CHANGES HERE

# Save to TSV
estimated_returns.to_csv("estimated_returns.tsv", sep="\t",
                         float_format="%.6f", index=False)

print("Saved estimated_returns.tsv")
print(estimated_returns)

Saved estimated_returns.tsv
   asset  estimated_return
0   AAPL          0.006383
1   MSFT          0.023842
2   AMZN          0.025579
3  GOOGL          0.017640
4    SPY          0.016712
5   TSLA          0.020378


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [14]:
# YOUR CHANGES HERE

cov_matrix = ret.cov()

Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [15]:
# YOUR CHANGES HERE

cov_matrix.to_csv("estimated_covariance.tsv", sep="\t", float_format="%.6f")

print("Saved estimated_covariance.tsv")
print(cov_matrix.round(6))

Saved estimated_covariance.tsv
           AAPL      MSFT      AMZN     GOOGL       SPY      TSLA
AAPL   0.003540  0.001019  0.001168  0.000595  0.001118  0.003199
MSFT   0.001019  0.003971  0.003005  0.001617  0.001675  0.002561
AMZN   0.001168  0.003005  0.004913  0.002852  0.001986  0.005961
GOOGL  0.000595  0.001617  0.002852  0.004793  0.000968  0.004414
SPY    0.001118  0.001675  0.001986  0.000968  0.001488  0.002574
TSLA   0.003199  0.002561  0.005961  0.004414  0.002574  0.026668


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [16]:
# YOUR CHANGES HERE

import pandas as pd

mu = ret.mean()                     
best = mu.idxmax()
w = pd.Series(0.0, index=mu.index)
w[best] = 1.0 

Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [17]:
# YOUR CHANGES HERE

max_port = w.rename("allocation").reset_index().rename(columns={"index":"asset"})
max_port.to_csv("maximum_return.tsv", sep="\t", index=False, float_format="%.6f")

print(max_port)
print("Sum of allocations:", w.sum())

   asset  allocation
0   AAPL         0.0
1   MSFT         0.0
2   AMZN         1.0
3  GOOGL         0.0
4    SPY         0.0
5   TSLA         0.0
Sum of allocations: 1.0


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [18]:
# YOUR CHANGES HERE

import cvxpy as cp, numpy as np, pandas as pd
Sigma, tickers = ret.cov().values, ret.columns
w = cp.Variable(len(tickers), nonneg=True)
cp.Problem(cp.Minimize(cp.quad_form(w, Sigma)), [cp.sum(w)==1]).solve()


np.float64(0.0013765261248789352)

Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [19]:
# YOUR CHANGES HERE

pd.DataFrame({"asset":tickers, "allocation":w.value}).to_csv("minimum_risk.tsv", sep="\t", index=False, float_format="%.6f")

Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [21]:
# YOUR CHANGES HERE

import numpy as np, pandas as pd, cvxpy as cp
mu, Sigma, tick = ret.mean().values, ret.cov().values, ret.columns
n = len(tick)

# min risk return
w = cp.Variable(n, nonneg=True)
cp.Problem(cp.Minimize(cp.quad_form(w,Sigma)), [cp.sum(w)==1]).solve()
rmin = mu @ w.value

# max return
rmax = mu.max()

# efficient frontier portfolios
rows=[]
for i,t in enumerate(np.linspace(rmin,rmax,101)):
    w = cp.Variable(n, nonneg=True)
    cp.Problem(cp.Minimize(cp.quad_form(w,Sigma)), [cp.sum(w)==1, mu@w>=t]).solve()
    wv = w.value/ w.value.sum()
    rows.append([i, mu@wv, np.sqrt(wv@Sigma@wv), *wv])


Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [22]:
# YOUR CHANGES HERE

pd.DataFrame(rows, columns=["index","return","risk",*tick]).to_csv(
    "efficient_frontier.tsv", sep="\t", index=False, float_format="%.6f")

Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [35]:
# YOUR CHANGES HERE

import numpy as np, pandas as pd

mu, Sigma, tick = ret.mean().values, ret.cov().values, ret.columns
n, T, B = len(tick), 23, 1000
Sigma = Sigma + 1e-12*np.eye(n)  

rng = np.random.default_rng(42)
wins = np.zeros(n, dtype=int)

for _ in range(B):
    samp = rng.multivariate_normal(mu, Sigma, size=T)  
    mu_b = samp.mean(axis=0)                            
    wins[np.argmax(mu_b)] += 1

probs = wins / B

Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [36]:
# YOUR CHANGES HERE

out = pd.DataFrame({"asset": tick, "probability": probs})
out.to_csv("max_return_probabilities.tsv", sep="\t", index=False, float_format="%.6f")
out

,asset,probability
0,AAPL,0.035
1,MSFT,0.218
2,AMZN,0.223
3,GOOGL,0.119
4,SPY,0.031
5,TSLA,0.374


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [34]:
with open("acknowledgments.txt","w") as f:
    f.write(conversation)

print("Saved acknowledgments.txt")


Saved acknowledgments.txt


https://www.youtube.com/watch?v=tjv9yYpvYc8
https://chatgpt.com/share/68b211ab-0e80-8007-8a77-505c3a145044


In [37]:
ack_text = """Acknowledgments

I used ChatGPT as a study partner to clarify concepts, check errors, and simplify code while completing this project. 
Conversations focused on understanding portfolio construction, covariance matrices, maximum return/minimum risk portfolios, 
efficient frontiers, and stability checks. The tool acted as a tutor/debugging assistant, not as a replacement for my work.

For transparency, I am including links to the transcripts of these interactions:

1. ChatGPT guidance: [https://chatgpt.com/share/68b211ab-0e80-8007-8a77-505c3a145044]
2. Youtube: [https://www.youtube.com/watch?v=tjv9yYpvYc8]

These resources document the support I received and ensure compliance with the class generative AI policy.
"""

with open("acknowledgments.txt","w") as f:
    f.write(ack_text)

print("Saved acknowledgments.txt")

Saved acknowledgments.txt


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.